In [16]:
import numpy as np
from tqdm import tqdm
num_replicates = 1000

def boostrap_replicate(y_pred, y_true, func):
    NUM_SAMPLES = np.shape(y_pred)[0]
    data = np.stack([y_pred, y_true], axis=0)
    rng = np.random.default_rng()
    bootstrap = rng.choice(data, size=NUM_SAMPLES, replace=True, axis=1)
    val = func(bootstrap[0], bootstrap[1])
    return val

def draw_bs_replicate(y_pred, y_true, func, num):
    vals = []
    for i in tqdm(range(num)):
        val = boostrap_replicate(y_pred, y_true, func)
        vals.append(val)
    return vals

In [17]:
# Load data
from helper_functions import *
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
base_dir = ''
matlab_file = base_dir + 'kamitani_data/fmri/Subject3.mat'
test_image_ids = base_dir + 'kamitani_data/images/image_test_id.csv'
train_image_ids = base_dir + 'kamitani_data/images/image_training_id.csv'
images_npz = base_dir + 'kamitani_data/images/images_112.npz'

x_train, x_test, y_train, y_test, xyz = load_data(matlab_file, test_image_ids, train_image_ids, images_npz, roi='ROI_VC')
NUM_VOXELS = y_train.shape[1]

digits_data = loadmat('handwritten_digits_in_fmri_dataset/69dataset_split.mat')
digits_test = digits_data['y_test']
digits_train =  digits_data['y_train']
scaler = StandardScaler()
digits_train = scaler.fit_transform(digits_train)
digits_test  = scaler.transform(digits_test)

# Imagenet model predictions
beliy_pred = np.loadtxt('models/beliy/gaziv_y_pred.csv', delimiter=',')
caps_pred = np.loadtxt('models/caps_encoder_test2/y_pred.csv', delimiter=',')
naive_pred = np.loadtxt('models/naive/y_pred.csv', delimiter=',')

#MNIST model predictions
digits_beliy_pred = np.loadtxt('models/digits_beliy/y_pred.csv', delimiter=',')
digits_naive_pred = np.loadtxt('models/naive/digits_pred.csv', delimiter=',')
digits_cnn_pred = np.loadtxt('models/digits_cnn_wide/y_pred.csv', delimiter=',')
digits_caps_pred = np.loadtxt('models/digits_encoder_test4/y_pred.csv', delimiter=',')

In [3]:
# Metric functions
# Data will be shape (2, num_samples, num_voxels)

def voxel_corr(y_pred, y_true):
  NUM_VOXELS = np.shape(y_pred)[1]
  vc = []
  for i in range(NUM_VOXELS):
    vc.append(stats.pearsonr(y_pred[:, i], y_true[:, i])[0]) 
  return np.array(vc)

def vc_mean(y_pred, y_true):
  return np.mean(voxel_corr(y_pred, y_true))

def vc_top100_mean(y_pred, y_true):
  vc = voxel_corr(y_pred, y_true)
  vc_sort = np.sort(vc)
  return np.mean(vc_sort[-100:])

def sample_corr(y_pred, y_true):
  NUM_SAMPLES = np.shape(y_pred)[0]
  sc = []
  for i in range(NUM_SAMPLES):
    sc.append(stats.pearsonr(y_pred[i], y_true[i])[0])
  return np.array(sc)

def sc_mean(y_pred, y_true):
  return np.mean(sample_corr(y_pred, y_true))

def response_sensitivity(y_pred, y_true):
  return np.std(y_pred, axis=0)

def rs_mean(y_pred, y_true):
  return np.mean(response_sensitivity(y_pred, y_true))

def response_weighted_corr(y_pred, y_true):
  vc = voxel_corr(y_pred, y_true)
  rs = response_sensitivity(y_pred, y_true)
  vcrs = vc * rs
  rwc = np.sign(vcrs)*np.sqrt(abs(vcrs))
  return rwc

def rwc_mean(y_pred, y_true):
  return np.mean(response_weighted_corr(y_pred, y_true))

def confidence_95(vals):
  z = 1.960 # z-score for a confidence of 95%
  n = len(vals)
  return z*np.std(vals)/(np.sqrt(n))

def mse(y_pred, y_true):
  diff = np.subtract(y_pred, y_true)
  return np.mean(np.square(diff))

def rwc_thresh(y_pred, y_true):
  rwc = response_weighted_corr(y_pred, y_true)
  return abs(np.min(rwc))

def get_int_vox_idx(y_pred, y_true):
  NUM_VOXELS = np.shape(y_pred)[1]
  rwc = response_weighted_corr(y_pred, y_true)
  thresh = abs(np.min(rwc))
  return np.where(rwc > thresh)[0]

def above_thresh_percentage(y_pred, y_true):
  v_int = get_int_vox_idx(y_pred, y_true)
  return len(v_int)*100/NUM_VOXELS

def rwc_skew(y_pred, y_true):
  vc = voxel_corr(y_pred, y_true)
  rs = response_sensitivity(y_pred, y_true)
  rho, phi = cart2pol(1 - rs, 1 - vc)
  skew = phi*(180/np.pi) - 45
  return skew

def rwc_skew_mean(y_pred, y_true):
  return np.mean(rwc_skew(y_pred, y_true))

In [18]:
# Calculate Normal Metrics for Imagenet-fMRI Encoders
import pickle
print('ImageNet-fMRI')
metrics = [mse, vc_mean, vc_top100_mean, sc_mean, rs_mean]
model_preds = [beliy_pred, caps_pred, naive_pred]
model_names = ['beliy', 'capsule', 'naive']
metric_names = ['MSE', 'VC', 'VS_Top100', 'SC', 'RS']
bagging_metrics_a = {'MSE': [], 'VC': [], 'VS_Top100': [], 'SC': [], 'RS': []}

for i, model_pred in enumerate(model_preds):
  print(model_names[i])
  for j, metric in enumerate(metrics):
    vals = draw_bs_replicate(model_pred, y_test, metric, num_replicates)
    mean = np.mean(vals)
    ci = confidence_95(vals)
    std = np.std(vals)
    print(metric_names[j], ': %f+-%f, %f, %f' % (mean, ci, std, 2*std))
    bagging_metrics_a[metric_names[j]].append(vals)

with open('imagenet_bag_metrics.pkl', 'wb') as f:
  pickle.dump(bagging_metrics_a, f)

ImageNet-fMRI
beliy


100%|██████████| 1000/1000 [00:02<00:00, 371.20it/s]


MSE : 0.062991+-0.000137, 0.002203, 0.004405


100%|██████████| 1000/1000 [05:23<00:00,  3.09it/s]


VC : 0.293305+-0.001588, 0.025618, 0.051236


...

VS_Top100 : 0.459931+-0.001343, 0.021665, 0.043329


100%|██████████| 1000/1000 [00:08<00:00, 119.83it/s]


SC : 0.159350+-0.000941, 0.015186, 0.030373


100%|██████████| 1000/1000 [00:02<00:00, 471.52it/s]


RS : 0.037904+-0.000013, 0.000213, 0.000427


In [59]:
# Calculate Normal Metrics for MNIST-fMRI Encoders
print('MNIST-fMRI')
metrics = [mse, vc_mean, vc_top100_mean, sc_mean, rs_mean]
model_preds = [digits_beliy_pred, digits_caps_pred, digits_cnn_pred, digits_naive_pred]
model_names = ['beliy', 'capsule', 'large_cnn', 'naive']
metric_names = ['MSE', 'VC', 'VS_Top100', 'SC', 'RS']
bagging_metrics_b = {'MSE': [], 'VC': [], 'VS_Top100': [], 'SC': [], 'RS': []}

for i, model_pred in enumerate(model_preds):
  print(model_names[i])
  for j, metric in enumerate(metrics):
    vals = draw_bs_replicate(model_pred, digits_test, metric, num_replicates)
    mean = np.mean(vals)
    ci = confidence_95(vals)
    std = np.std(vals)
    print(metric_names[j], ': %f+-%f, %f, %f' % (mean, ci, std, 2*std))
    bagging_metrics_b[metric_names[j]].append(vals)

with open('mnist_bag_metrics.pkl', 'wb') as f:
  pickle.dump(bagging_metrics_b, f)
    

MNIST-fMRI
beliy


100%|██████████| 1000/1000 [00:00<00:00, 1205.42it/s]


MSE : 1.046838+-0.003408, 0.054982, 0.109964


100%|██████████| 1000/1000 [03:55<00:00,  4.24it/s]


VC : 0.145338+-0.001688, 0.027235, 0.054471


...

VS_Top100 : 0.646211+-0.002343, 0.037799, 0.075599


100%|██████████| 1000/1000 [00:02<00:00, 359.72it/s]


SC : -0.000076+-0.000657, 0.010607, 0.021215


100%|██████████| 1000/1000 [00:00<00:00, 1482.47it/s]


RS : 0.035702+-0.000037, 0.000595, 0.001190


In [12]:
# Calculate new metrics for imagenet fMRI
print('New Metrics ImageNet-fMRI')
b_int = get_int_vox_idx(beliy_pred, y_test)
c_int = get_int_vox_idx(caps_pred, y_test)
n_int = get_int_vox_idx(naive_pred, y_test)

model_preds = [beliy_pred, caps_pred]
voxel_select = [b_int, c_int]
model_names = ['beliy', 'capsule']
bagging_metrics_c = {'rwc_skew': [], 'rwc': [], 'vc': [], 'rs': []}

metrics = [rwc_skew_mean]
metric_names = ['rwc_skew']
for i, model_pred in enumerate(model_preds):
  print(model_names[i])
  for j, metric in enumerate(metrics):
    select = voxel_select[j]
    vals = draw_bs_replicate(model_pred[:, select], y_test[:, select], metric, num_replicates)
    mean = np.mean(vals)
    ci = confidence_95(vals)
    std = np.std(vals)
    print(metric_names[j], ': %f+-%f, %f, %f' % (mean, ci, std, 2*std))
    bagging_metrics_c[metric_names[j]].append(vals)


model_preds = [beliy_pred, caps_pred, naive_pred]
model_names = ['beliy', 'capsule', 'naive']
metrics = [rwc_mean, vc_mean, rs_mean]
metric_names = ['rwc', 'vc', 'rs']
v_int = np.union1d(b_int, c_int).astype(int)
v_int = np.union1d(v_int, n_int).astype(int)

for i, model_pred in enumerate(model_preds):
  print(model_names[i])
  for j, metric in enumerate(metrics):
    vals = draw_bs_replicate(model_pred[:, v_int], y_test[:, v_int], metric, num_replicates)
    mean = np.mean(vals)
    ci = confidence_95(vals)
    std = np.std(vals)
    print(metric_names[j], ': %f+-%f, %f, %f' % (mean, ci, std, 2*std))
    bagging_metrics_c[metric_names[j]].append(vals)

with open('imagenet_bag_new_metrics.pkl', 'wb') as f:
  pickle.dump(bagging_metrics_c, f)

New Metrics ImageNet-fMRI
beliy


100%|██████████| 1000/1000 [02:20<00:00,  7.13it/s]


rwc_skew : -17.981615+-0.093876, 1.514606, 3.029211
capsule


100%|██████████| 1000/1000 [02:20<00:00,  7.11it/s]


rwc_skew : -3.747018+-0.096234, 1.552648, 3.105297
beliy


...

rwc : 0.001425+-0.000087, 0.001412, 0.002823


100%|██████████| 1000/1000 [02:22<00:00,  7.00it/s]


vc : 0.003572+-0.000224, 0.003622, 0.007243


100%|██████████| 1000/1000 [00:01<00:00, 579.10it/s]


rs : 0.038087+-0.000017, 0.000275, 0.000550


In [13]:
print(rwc_thresh(naive_pred, y_test))
print(above_thresh_percentage(naive_pred, y_test))

print(rwc_thresh(digits_naive_pred, digits_test))
print(above_thresh_percentage(digits_naive_pred, digits_test))

0.1549588450355794
0.0
0.19372811233415888
0.0


In [25]:
# Calculate new metrics for imagenet fMRI
print('New Metrics MNIST-fMRI')
b_int = get_int_vox_idx(digits_beliy_pred, digits_test)
c_int = get_int_vox_idx(digits_caps_pred, digits_test)
n_int = get_int_vox_idx(digits_naive_pred, digits_test)
cnn_int = get_int_vox_idx(digits_cnn_pred, digits_test)

model_preds = [digits_beliy_pred, digits_caps_pred, digits_cnn_pred]
voxel_select = [b_int, c_int, cnn_int]
model_names = ['beliy', 'capsule', 'large_cnn']
bagging_metrics_d = {'rwc_skew': [], 'rwc': [], 'vc': [], 'rs': []}

metrics = [rwc_skew_mean]
metric_names = ['rwc_skew']
for i, model_pred in enumerate(model_preds):
  print(model_names[i])
  for j, metric in enumerate(metrics):
    select = voxel_select[j]
    vals = draw_bs_replicate(model_pred[:, select], digits_test[:, select], metric, num_replicates)
    mean = np.mean(vals)
    ci = confidence_95(vals)
    std = np.std(vals)
    print(metric_names[j], ': %f+-%f, %f, %f' % (mean, ci, std, 2*std))
    bagging_metrics_d[metric_names[j]].append(vals)

v_int = np.union1d(b_int, c_int).astype(int)
v_int = np.union1d(v_int, cnn_int).astype(int)
v_int = np.union1d(v_int, n_int).astype(int)

model_preds = [digits_beliy_pred, digits_caps_pred, digits_cnn_pred, digits_naive_pred]
model_names = ['beliy', 'capsule', 'large_cnn', 'naive']
metrics = [rwc_mean, vc_mean, rs_mean]
metric_names = ['rwc', 'vc', 'rs']
for i, model_pred in enumerate(model_preds):
  print(model_names[i])
  for j, metric in enumerate(metrics):
    vals = draw_bs_replicate(model_pred[:, v_int], digits_test[:, v_int], metric, num_replicates)
    mean = np.mean(vals)
    ci = confidence_95(vals)
    std = np.std(vals)
    print(metric_names[j], ': %f+-%f, %f, %f' % (mean, ci, std, 2*std))
    bagging_metrics_d[metric_names[j]].append(vals)

with open('mnist_bag_new_metrics.pkl', 'wb') as f:
  pickle.dump(bagging_metrics_a, f)

New Metrics MNIST-fMRI
beliy


100%|██████████| 1000/1000 [00:38<00:00, 26.16it/s]


rwc_skew : -15.166420+-0.146990, 2.371552, 4.743104
capsule


100%|██████████| 1000/1000 [00:37<00:00, 26.67it/s]


rwc_skew : -6.968781+-0.145698, 2.350705, 4.701410
large_cnn


...

rwc : 0.002046+-0.000168, 0.002713, 0.005427


100%|██████████| 1000/1000 [00:57<00:00, 17.28it/s]


vc : 0.006046+-0.000482, 0.007784, 0.015568


100%|██████████| 1000/1000 [00:00<00:00, 2854.64it/s]


rs : 0.035362+-0.000042, 0.000683, 0.001366


In [58]:
from scipy.stats import wilcoxon

print(bagging_metrics_a.keys())
print(bagging_metrics_d.keys())
print(np.shape(bagging_metrics_d['rwc']))
print(np.shape(bagging_metrics_d['rwc'][0]))
print(np.mean(bagging_metrics_d['rwc'][0]))
print(np.mean(bagging_metrics_d['rwc'][1]))
w_stats = wilcoxon(x=bagging_metrics_d['rwc'][1],  y=bagging_metrics_d['rwc'][2], alternative='greater')
print(w_stats.statistic, w_stats.pvalue)
w_stats = wilcoxon(x=bagging_metrics_d['vc'][1],  y=bagging_metrics_d['vc'][2], alternative='greater')
print(w_stats.statistic, w_stats.pvalue)
w_stats = wilcoxon(x=bagging_metrics_d['rs'][1],  y=bagging_metrics_d['rs'][2], alternative='greater')
print(w_stats.statistic, w_stats.pvalue)

dict_keys(['MSE', 'VC', 'VS_Top100', 'SC', 'RS'])
dict_keys(['rwc_skew', 'rwc', 'vc', 'rs'])
(4, 1000)
(1000,)
0.27650382269654405
0.4762086817466359
460866.0 6.622924907708811e-118
260179.0 0.13855103399783902
497442.0 1.5278992350307103e-161
